# Elektrisches Feld im Faradaykäfig

Siehe Wiedemann/Ingold: *Numerische Physik mit Python*, Springer-Spektrum 2024, ISBN 978-3-662-69566-1

---

Das elektrische Feld sowie das elektrische Potential einer Punktladung in einem quaderförmigen Faradaykäfig mit leitenden Wänden lassen sich mit Hilfe der Methode der Spiegelladungen berechnen, wobei allerdings streng genommen über eine unendliche Anzahl von Spiegelladungen zu summieren ist. Da der Beitrag der Spiegelladungen mit dem Abstand vom Faradaykäfig abnimmt, kann man sich näherungsweise auf endlich viele Spiegelladungen beschränken. Auf diese Weise wird in diesem Jupyter-Notebook das elektrische Feld und das zugehörige Potential berechnet und graphisch dargestellt.

## Importanweisungen

In [ ]:
import numpy as np
import numpy.linalg as LA
import ipywidgets as widgets
from ipywidgets import interact_manual
import matplotlib.pyplot as plt

plt.style.use("numphyspy.style")

## Berechnung des elektrischen Feldes und des Potentials einer Punktladung

Die Funktionen `e_point` und `v_point` berechnen das elektrische Feld $\boldsymbol{E}$ bzw. das elektrische Potential $\Phi$ am Ort $\boldsymbol{r}$ einer Einheitspunktladung, die sich am Ort $\boldsymbol{r}_0$ befindet. Dabei können die Argumente `r`und `r0` durch Arrays gegeben sein. Die drei Ortskomponenten müssen dann entlang der Achse 1 liegen.

In [ ]:
def e_point(r, r0):
    distance = LA.norm(r-r0, axis=1)[..., np.newaxis]
    return (r-r0)/distance**3

In [ ]:
def v_point(r, r0):
    distance = LA.norm(r-r0, axis=1)[..., np.newaxis]
    return -1/distance

## Berechnung der Spiegelladungen und ihrer Orte

Die Funktion `image_charges` berechnet alle Variablen, die zur Modellierung der Punktladung inklusive aller Spiegelladungen erforderlich sind. `r_image_charges` ist ein Array, das die Orte der Ladungen in der Elementarzelle enthält, die sich in $x$-Richtung von $-l_x$ bis $l_x$ und entsprechend in $y$-Richtung von $-l_y$ bis $l_y$ sowie in $z$-Richtung von $-l_z$ bis $l_z$ erstreckt. Das Array `q_image_charges` enthält die dazugehörigen Einheitsladungen -1 bzw. +1. Das Array `delta_image` enthält die Verschiebungen der Elementarzelle, mit der sich die anderen zu berücksichtigenden Elementarzellen ergeben.

In [ ]:
def image_charges(lx, ly, lz, r0, n_max):
    mirroring = np.array([[1, 1, 1],
                          [1, 1, -1],
                          [1, -1, 1],
                          [1, -1, -1],
                          [-1, 1, 1],
                          [-1, 1, -1],
                          [-1, -1, 1],
                          [-1, -1, -1]])
    r_image_charges = r0*mirroring
    q_image_charges = np.prod(mirroring, axis=1)
    n_idx = np.mgrid[-n_max:n_max+1,
                     -n_max:n_max+1,
                     -n_max:n_max+1]
    n_idx = np.moveaxis(n_idx, 0, 3).reshape(-1, 3)
    delta_image = 2*np.array([lx, ly, lz])*n_idx
    return r_image_charges, q_image_charges, delta_image

## Berechnung des elektrischen Feldes und des Potentials an einem Punkt

Die Funktionen `e_field_point` und `v_field_point` berechnen das elektrische Feld bzw. das Potential am Ort $\boldsymbol{r}$ durch Summation über die Beiträge der 
Punktladungen. Die hierzu benötigten Argumente `r_image_charges`, `q_image_charges` und `delta_image` können mit Hilfe der Funktion `image_charges` berechnet werden.

In [ ]:
def e_field_point(r, r_image_charges, q_image_charges,
                  delta_image):
    e = 0
    for r0, q in zip(r_image_charges, q_image_charges):
        e = e + q*np.sum(e_point(r, r0+delta_image),
                         axis=0)
    return e

In [ ]:
def v_field_point(r, r_image_charges, q_image_charges,
                  delta_image):
    v = 0
    for r0, q in zip(r_image_charges, q_image_charges):
        v = v + q*np.sum(v_point(r, r0+delta_image))
    return v

## Berechnung des gesamten elektrischen Feldes und des Potentials

Nun werden die zuvor definierten Funktionen verwendet, um das elektrische Feld und das elektrische Potential auf dem gesamten Gitter zu berechnen. Dabei unterscheiden sich die beiden Funktionen `e_field` und `v_field` lediglich in der Form des Arrays `efield` bzw. `vfield` sowie in der Funktion `e_field_point` bzw. `v_field_point` zur Berechnung an einem einzelnen Punkt.

Zunächst werden mit der Funktion `image_charges` die benötigten Parameter der Bildladungen beschafft. Das Gitter, auf dem elektrisches Feld und Potential zu berechnen sind, wird durch die Arrays `xvals` und `yvals` festgelegt. In der Doppelschleife werden dann die zugehörigen Feld- bzw. Potentialwerte auf dem sich in $x$-$y$-Richtung erstreckenden Gitter auf halber Höhe in $z$-Richtung berechnet. 

In [ ]:
def e_field(lx, ly, lz, r0, n_max, n_out):
    images_data = image_charges(lx, ly, lz, r0, n_max)
    efield = np.empty((3, n_out, n_out))
    yvals, xvals = np.mgrid[0:ly:n_out*1j, 0:lx:n_out*1j]
    for nx in range(n_out):
        for ny in range(n_out):
            efield[:, ny, nx] = e_field_point(
                np.array([xvals[ny, nx],
                          yvals[ny, nx],
                          0.5*lz]),
                *images_data)
    return xvals, yvals, efield[0], efield[1], efield[2]

In [ ]:
def v_field(lx, ly, lz, r0, n_max, n_out):
    images_data = image_charges(lx, ly, lz, r0, n_max)
    vfield = np.empty((n_out, n_out))
    yvals, xvals = np.mgrid[0:ly:n_out*1j, 0:lx:n_out*1j]
    for nx in range(n_out):
        for ny in range(n_out):
            vfield[ny, nx] = v_field_point(
                np.array([xvals[ny, nx],
                          yvals[ny, nx],
                          0.5*lz]),
                *images_data)
    return xvals, yvals, vfield

## Implementierung der Bedienelemente und graphische Darstellung des elektrischen Feldes und des Potentials

Mit Hilfe der Schieberegler lassen sich die folgenden Parameter einstellen:
- `lx`: Ausdehnung des Faradaykäfigs in $x$-Richtung
- `ly`: Ausdehnung des Faradaykäfigs in $y$-Richtung
- `lz`: Ausdehnung des Faradaykäfigs in $z$-Richtung
- `r0_x`: Position der Punktladung in Einheiten von `lx`
- `r0_y`: Position der Punktladung in Einheiten von `ly`
- `r0_z`: Position der Punktladung in Einheiten von `lz`
- `n_max`: legt die Gesamtzahl der zu berücksichtigenden Ladungen $8(2n_\text{max}+1)^3$ fest
- `n_out`: Anzahl der Gitterpunkte in $x$- und in $y$-Richtung auf denen das Feld und das Potential zu berechnen sind
- `alpha`: Parameter für die Empfindlichkeit der Farbe auf Potentialänderung in der Nähe des Potentialnullpunkts

Das elektrische Feld wird mit Hilfe der matplotlib-Funktion `streamplot` durch Feldlinien dargestellt. Hinterlegt ist eine farbliche Darstellung des elektrischen Potentials.

In [ ]:
widget_dict = {"lx":
               widgets.FloatSlider(
                   value=2, min=0, max=5, step=0.1,
                   description="$l_x$"),
               "ly":
               widgets.FloatSlider(
                   value=1, min=0, max=5, step=0.1,
                   description="$l_y$"),
               "lz":
               widgets.FloatSlider(
                   value=1, min=0, max=5, step=0.1,
                   description="$l_z$"),
               "r0_x":
               widgets.FloatSlider(
                   value=0.8, min=0, max=1, step=0.05,
                   description="$x_0$"),
               "r0_y":
               widgets.FloatSlider(
                   value=0.7, min=0, max=1, step=0.05,
                   description="$y_0$"),
               "r0_z":
               widgets.FloatSlider(
                   value=0.5, min=0, max=1, step=0.05,
                   description="$z_0$"),
               "n_max":
               widgets.IntSlider(
                   value=10, min=1, max=20, step=1,
                   description=r"$n_\text{max}$"),
               "n_out":
               widgets.IntSlider(
                   value=60, min=5, max=100, step=5,
                   description=r"$n_\text{out}$"),
               "alpha":
               widgets.FloatSlider(
                   value=2, min=1, max=10, step=1,
                   description=r"$\alpha$")
               }

interact_start = interact_manual.options(
    manual_name="Start Berechnung")

@interact_start(**widget_dict)
def plot_result(lx, ly, lz, r0_x, r0_y, r0_z, n_max, n_out,
                alpha):
    r0 = np.array([r0_x*lx, r0_y*ly, r0_z*lz])
    args = lx, ly, lz, r0, n_max, n_out
    xvals, yvals, vfield = v_field(*args)

    fig, ax = plt.subplots()
    ax.set_aspect("equal")
    ax.pcolormesh(xvals, yvals, np.arctan(alpha*vfield),
                  shading="auto")
    ax.streamplot(*e_field(*args)[:4])
    ax.set_xlabel("$x$")
    ax.set_ylabel("$y$")